In [44]:
import numpy as np 
import torch

batch_size = 5


At first we have : 
- camera's parameters -> A (focal, center) 
- rotation matrix  -> R 
- position matrix -> C 
- 3D points position ->  P1 P2 P3 (and P4 to determinate the best solution after P3P)

In [45]:
# This script defines the camera parameters, rotation matrix, and translation matrix.
def camera() : 
  # Definition of the camera parameters
  # focal length
  fx = 800
  fy = 800
  # center
  cx = 320 
  cy = 240

  A = torch.tensor([[fx, 0, cx], [0, fy, cy], [0, 0, 1]], dtype=torch.float64) # intraseca matrix of the camera (3*3)
  
  A_batch = A.repeat(batch_size,1,1)
  print("A_batch = \n", A_batch)
  print(A_batch.shape)  # (batch_size, 3, 3)
  return A_batch

A = camera() 



def rotation_matrix() : 
  # Definition of the rotation matrix of the camera 
  R = torch.tensor([[1, 0, 0],[0, -1, 0], [0, 0, -1]], dtype=torch.float64)  # (3*3)
  
  R_batch = R.repeat(batch_size,1,1)  # Repeat the rotation matrix for each batch
  print("R_batch = \n", R_batch)  
  print(R_batch.shape)  # (batch_size, 3, 3)
  return R_batch

def camera_position() : 
  # Definition of the translation matrix of the camera (the position)
  C = torch.tensor([[0,0,6]], dtype=torch.float64)    # T = [tx,ty,tz]  (1*3)
  

  C_batch = C.repeat(batch_size, 1)  # Repeat the translation vector for each batch
  print("C_batch = \n", C_batch)  
  print(C_batch.shape)  # (batch_size, 3)
  return C_batch

R = rotation_matrix()
C = camera_position()
print("batch_size = ", batch_size)


A_batch = 
 tensor([[[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]],

        [[800.,   0., 320.],
         [  0., 800., 240.],
         [  0.,   0.,   1.]]], dtype=torch.float64)
torch.Size([5, 3, 3])
R_batch = 
 tensor([[[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]],

        [[ 1.,  0.,  0.],
         [ 0., -1.,  0.],
         [ 0.,  0., -1.]]], dtype=torch.float64)
torch.Size([5, 3, 3])
C_batch = 
 tensor([[0., 0., 6.],
        [0.

In [46]:
#torch.manual_seed(1234)  

# Definition of 3D points in the world coordinate system
def point3Daleatoire(x) :
  # Generation of one random points in 3D space 
  return torch.empty((1, 3), dtype=torch.float64).uniform_(-x, x)


def pts_3D_4pts():
  # Generate randomly 4 3D points
  # Output : tensor which concatenate the 4 points = [ P1, P2, P3, P4 ] 

  P1 = point3Daleatoire(2)     # (1*3) -> pour P3P
  P2 = point3Daleatoire(2)
  P3 = point3Daleatoire(2)
  P4 = point3Daleatoire(2)
  
  points3D = torch.cat((P1,P2,P3,P4),dim=0);     # (LIGNES 4* COLONNES 3) - xyz
    
  return points3D

def pts_3D_4pts_batch():
  # Generate randomly 4 3D points for each batch
  # Output : array which concatenate the 4 points = [ P1, P2, P3, P4 ] for each batch

  # Generate a batch of random points in 3D space
  # Each point is generated independently for each batch

  points3D_batch = torch.stack([pts_3D_4pts() for i in range(batch_size)])  # (batch_size, 4, 3)
  
  return points3D_batch

points3D_batch = pts_3D_4pts_batch()  # Generate the batch of 3D points
'''
P1 = torch.tensor([0.7161, 0.5431, 1.7807], dtype=torch.float64)    # (3,)
P2 = torch.tensor([-1.1643, 0.8371, -1.0551], dtype=torch.float64)
P3 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64)
P4 = torch.tensor([-1.5224, 0.4292, -0.1994], dtype=torch.float64) 
'''
P1 = points3D_batch[:, 0, :]  # Extract P1 for each batch
P2 = points3D_batch[:, 1, :]  # Extract P2 for each batch
P3 = points3D_batch[:, 2, :]  # Extract P3 for each batch
P4 = points3D_batch[:, 3, :]  # Extract P4 for each batch

print("P1 = \n", P1)
print("P2 = \n", P2)
print("P3 = \n", P3)
print("P4 = \n", P4)
print(P1.shape)  # (batch_size, 3)


P1 = 
 tensor([[ 0.3122, -0.3304, -1.1310],
        [-1.2820,  1.0684,  0.9066],
        [ 1.8376,  0.8464, -0.4832],
        [-0.1271, -1.1935,  0.1360],
        [-1.3699,  1.7906, -1.4741]], dtype=torch.float64)
P2 = 
 tensor([[ 1.5615, -1.5677, -0.9845],
        [ 0.5122, -0.7037, -1.1977],
        [-0.5159, -1.0735, -0.5221],
        [-1.8130, -1.3466, -1.6769],
        [-1.1599,  0.8379,  0.2833]], dtype=torch.float64)
P3 = 
 tensor([[ 0.1092, -1.3242, -1.3047],
        [ 1.4935,  0.3473,  0.1533],
        [ 0.7285,  1.1380,  0.7307],
        [-0.7767,  0.8582, -1.9929],
        [ 0.7726,  1.0623, -1.3456]], dtype=torch.float64)
P4 = 
 tensor([[-0.4773,  0.7125,  0.0666],
        [ 1.3850,  0.6512, -0.2771],
        [-1.6615, -1.8119, -0.0772],
        [-1.3723, -0.1566,  1.2420],
        [-1.7058,  0.7708, -0.5724]], dtype=torch.float64)
torch.Size([5, 3])


We create the 3 direction features vectors f1, f2, f3

In [47]:
def features_vectors(points3D,C, R,batch_size) :
    '''
    This function computes the features vectors for P3P algorithm.
    args:
    points3D : array with the 4 3D points = [ P1, P2, P3, P4 ] (4*3) 
    but we only use the first three points for P3P
    C: camera position matrix : (3*1)
    returns:
    featuresVect : array with the features vectors (9*1)
    '''
    P1 = torch.reshape(points3D[0], (batch_size,3,1))  # (batch_size, 3, 1)
    P2 = torch.reshape(points3D[1], (batch_size,3,1))
    P3 = torch.reshape(points3D[2], (batch_size,3,1))

    C = torch.reshape(C, (batch_size,3,1))   # (batch_size, 3, 1)
    
    v1 = torch.matmul(R,(P1 - C))  # (batch_size, 3, 1)
    v2 = torch.matmul(R,(P2 - C))  # (batch_size, 3, 1)
    v3 = torch.matmul(R,(P3 - C))  # (batch_size, 3, 1)

    f1 = v1 / torch.norm(v1,dim=1, keepdim=True)  # (batch_size, 3, 1)
    f2 = v2 / torch.norm(v2,dim=1, keepdim=True)  # (batch_size, 3, 1)
    f3 = v3 / torch.norm(v3,dim=1, keepdim=True)  # (batch_size, 3, 1)


    f1 = torch.reshape(f1, (batch_size,1,3))  # (batch_size,1,3)
    f2 = torch.reshape(f2, (batch_size,1,3))
    f3 = torch.reshape(f3, (batch_size,1,3))

    featuresVect = torch.cat((f1,f2,f3),dim=1) # (batch_size, 3, 3)
    print("features vectors = \n",featuresVect)
    print(featuresVect.shape)  

    return featuresVect # Return the features vectors need in P3P


points3D = [P1, P2, P3]  # We define the points3D with the first three points
print("points3D = \n", points3D)  # Print the points3D to check the values  / List len = 3 

featuresVect = features_vectors(points3D, C, R,batch_size)  

points3D = 
 [tensor([[ 0.3122, -0.3304, -1.1310],
        [-1.2820,  1.0684,  0.9066],
        [ 1.8376,  0.8464, -0.4832],
        [-0.1271, -1.1935,  0.1360],
        [-1.3699,  1.7906, -1.4741]], dtype=torch.float64), tensor([[ 1.5615, -1.5677, -0.9845],
        [ 0.5122, -0.7037, -1.1977],
        [-0.5159, -1.0735, -0.5221],
        [-1.8130, -1.3466, -1.6769],
        [-1.1599,  0.8379,  0.2833]], dtype=torch.float64), tensor([[ 0.1092, -1.3242, -1.3047],
        [ 1.4935,  0.3473,  0.1533],
        [ 0.7285,  1.1380,  0.7307],
        [-0.7767,  0.8582, -1.9929],
        [ 0.7726,  1.0623, -1.3456]], dtype=torch.float64)]
features vectors = 
 tensor([[[ 0.0437,  0.0462,  0.9980],
         [ 0.2131,  0.2140,  0.9533],
         [ 0.0147,  0.1783,  0.9839]],

        [[-0.2392, -0.1993,  0.9503],
         [ 0.0706,  0.0971,  0.9928],
         [ 0.2471, -0.0575,  0.9673]],

        [[ 0.2706, -0.1246,  0.9546],
         [-0.0778,  0.1619,  0.9837],
         [ 0.1339, -0.2092,  0.96

Lastly we need the functions to resolve the polynomial roots. - for test go to test resolution polynome 

In [48]:
from complex_batch_utils import *
import complex_utils as cpu
from torch import vmap

def polynomial_root_calculation_3rd_degree(a, b, c, d):
    # This function calculates the roots of a cubic polynomial of the form:
    # a*x^3 + b*x^2 + c*x + d = 0
        # a (batch_size, 1)
        # b (batch_size, 1)
        # c (batch_size, 1)
        # d (batch_size, 1)

    # output: roots of the polynomial in the form of a tensor of shape (batch_size,3, 2)
    # where each root is represented as a complex number (real, imaginary)
    # each row is the i_th root of the polynomial
    
    batch_size = a.shape[0]  # Get the batch size from the shape of a
    

    # Discriminant terms
    p = (3 * a * c - b**2) / (3 * a**2)     # (batch_size, 1) because element-wise opeations
    q = (2 * b**3 - 9 * a * b * c + 27 * a**2 * d) / (27 * a**3)    # (batch_size, 1)
    delta = -4 * p**3 - 27 * q**2   # (batch_size, 1)

    roots = torch.empty((batch_size,3, 2))  # Initialize roots tensor to store the roots

    j_ = torch.tensor([-0.5, torch.sqrt(torch.tensor(3))/2])  # cube root of unity
    
    for k in range (3):
        delta_sur_27 = -delta / 27   #(batch_size, 1) 

        sqrt_term = sqrt_batch(delta_sur_27)  

        # faire une seule fois les calculs de j^k et j^-k
        j_exp_k = cpu.complex_number_power_k(j_, k)  # Compute j^k for each batch
        j_exp_moins_k = cpu.complex_number_power_k(j_, -k)  # Compute j^-k for each batch

        j_exp_k_batch = j_exp_k.repeat(batch_size, 1)
        j_exp_moins_k_batch = j_exp_moins_k.repeat(batch_size, 1)

        u_k = product_of_2_complex_numbers_batch(j_exp_k_batch, sqrt_3_batch(torch.stack([0.5*(-q.squeeze()+sqrt_term[:,0]),sqrt_term[:,1]],dim=-1)) )
         # (batch_size, 2) 
        v_k = product_of_2_complex_numbers_batch(j_exp_moins_k_batch, sqrt_3_batch(torch.stack([0.5*(-q.squeeze()-sqrt_term[:,0]),-0.5*sqrt_term[:,1]],dim=-1)))
          # (batch_size, 2) 

        root = addition_batch(addition_batch(u_k, v_k), torch.stack([-b[:,0]/(3*a[:,0]),0.0*b[:,0]],dim=-1) ) 
         # (batch_size, 2)

        roots[:,k,:] = root  # Store the root in the roots tensor

    return roots


In [49]:

def polynomial_root_calculation_4th_degree_ferrari(a0, a1, a2, a3, a4): # Ferrari's Method
       
    # Solving a polynomial of 4th degree
    # a0, a1, a2, a3, a4 (batch_size,1)

    # Output : roots of the polynomial a4*x^4 + a3*x^3 + a2*x^2 + a1*x + a0   (4,batch_size 2) !! ATTENTION !!! 

    batch_size = a0.shape[0]  # Get the batch size from the shape of a0

    # Reduce the quartic equation to the form : x^4 + a*x^3 + b*x^2 + c*x + d = 0
    a = a3/a4           # (batch_size, 1)
    b = a2/a4
    c = a1/a4
    d = a0/a4

    # Computation of the coefficients of the Ferrari's Method
    S = a/4     # (batch_size, 1)
    b0 = d - c*S + b* S**2 - 3* S**4    # (batch_size, 1)
    b1 = c - 2*b*S + 8*S**3 # (batch_size, 1)
    b2 = b - 6 * S**2  # (batch_size, 1)


    # Solve the cubic equation m^3 + b2*m^2 + (b2^2/4  - b0)*m - b1^2/8 = 0
    x_cube = polynomial_root_calculation_3rd_degree(torch.tensor(1).repeat(b2.shape),b2,(b2**2)/4-b0,(-b1**2)/8)
  
    
    x_cube_real = x_cube[:,:,0] #  (batch_size, 3)
    x_cube_imag = x_cube[:,:,1]  #    (batch_size, 3)

    is_real = torch.isclose(x_cube_imag,torch.tensor(0.0))
    is_positive = x_cube[:,:,0] > 0
    condition = is_real & is_positive  # Condition to check if the root is real and positive   (batch_size, 3)
   
    
    real_filtered = x_cube_real.clone()
    real_filtered[~condition] = float('inf')  # if root real and positive, keep it, else set to infinity   (batch_size, 3)


    alpha_0_real, _ = real_filtered.min(dim=1) # Get the minimum real part of the roots (if doesn't exist, returns inf)

    alpha_0 = torch.stack([alpha_0_real, torch.zeros(batch_size)], dim=-1)  # (batch_size,2)
    

    # do the calculation for alpha_0_nul and not alpha_0_nul and then affects the good value 

    # if alpha_0_nul == False
    alpha0_div_2 = 0.5*alpha_0              # beacause alpha_0 is real  # (batch_size, 2)    
    sqrt_alpha = sqrt_batch(alpha0_div_2[:,0].unsqueeze(-1))    # input : (batch_size, 1) // output : (batch_size, 2)
    term = addition_complex_real_batch(- alpha0_div_2 ,-b2 / 2)      # (batch_size, 2)
    denom = 2 * torch.sqrt(2 * alpha_0)      # beacause alpha_0 is real  # (batch_size, 2)
    num = torch.stack([b1, torch.zeros(batch_size,1)], dim=-1).squeeze(1)  # (batch_size, 2)
   

    frac = division_2_complex_numbers(num,denom)    # (batch_size, 2)
    
    x1_false = addition_complex_real_batch(sqrt_alpha ,- S) + sqrt_complex_batch(addition_batch(term,-frac))    # (batch_size, 2)
    x2_false = addition_complex_real_batch(sqrt_alpha, - S) - sqrt_complex_batch(addition_batch(term,-frac))    # (batch_size, 2)
    x3_false = addition_complex_real_batch(-sqrt_alpha, - S) + sqrt_complex_batch(addition_batch(term,frac))    # (batch_size, 2)
    x4_false = addition_complex_real_batch(-sqrt_alpha,- S) - sqrt_complex_batch(addition_batch(term,frac))     # (batch_size, 2)
    
    # if alpha_0_nul == True
    sqrt_inner1 = sqrt_batch((b2**2) / 4 - b0)        # complex 

    x1_true = addition_complex_real_batch(sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1,-b2 / 2)),-S)
    x2_true = addition_complex_real_batch(- sqrt_complex_batch(addition_complex_real_batch(sqrt_inner1,-b2 / 2)),-S)
    x3_true = addition_complex_real_batch(sqrt_complex_batch(addition_complex_real_batch(- sqrt_inner1,-b2 / 2 )),-S)
    x4_true = addition_complex_real_batch(- sqrt_complex_batch(addition_complex_real_batch(- sqrt_inner1,-b2 / 2 )),-S)
    
    result = torch.where(alpha_0==float('inf'),
                         torch.stack([x1_true, x2_true, x3_true, x4_true]),torch.stack([x1_false, x2_false, x3_false, x4_false]))
    
    return result   # (4,batch_size, 2)

a0  = torch.rand(batch_size, 1, dtype=torch.float64)
a1  = torch.rand(batch_size, 1, dtype=torch.float64)  
a2  = torch.rand(batch_size, 1, dtype=torch.float64)  
a3  = torch.rand(batch_size, 1, dtype=torch.float64) 
a4  = torch.rand(batch_size, 1, dtype=torch.float64)  


We have all the variables needed for the p3p so we start

1. Storage of points : already done 

In [50]:
print("P1 = \n", P1)
print("P2 = \n", P2)
print("P3 = \n", P3)

P1 = 
 tensor([[ 0.3122, -0.3304, -1.1310],
        [-1.2820,  1.0684,  0.9066],
        [ 1.8376,  0.8464, -0.4832],
        [-0.1271, -1.1935,  0.1360],
        [-1.3699,  1.7906, -1.4741]], dtype=torch.float64)
P2 = 
 tensor([[ 1.5615, -1.5677, -0.9845],
        [ 0.5122, -0.7037, -1.1977],
        [-0.5159, -1.0735, -0.5221],
        [-1.8130, -1.3466, -1.6769],
        [-1.1599,  0.8379,  0.2833]], dtype=torch.float64)
P3 = 
 tensor([[ 0.1092, -1.3242, -1.3047],
        [ 1.4935,  0.3473,  0.1533],
        [ 0.7285,  1.1380,  0.7307],
        [-0.7767,  0.8582, -1.9929],
        [ 0.7726,  1.0623, -1.3456]], dtype=torch.float64)


2. Storage of the features vectors : done

In [51]:
# we got featuresVect and we access the 3 values 
f1 = featuresVect[:,0,:]  # Access the first feature vector for each batch
f2 = featuresVect[:,1,:]
f3 = featuresVect[:,2,:]

print("f1 = ", f1)
print(f1.shape)  # (batsh_size,3)
print("f2 = ", f2)
print(f2.shape)  # (batsh_size,3)
print("f3 = ", f3)
print(f3.shape)  # (batsh_size,3)

f1 =  tensor([[ 0.0437,  0.0462,  0.9980],
        [-0.2392, -0.1993,  0.9503],
        [ 0.2706, -0.1246,  0.9546],
        [-0.0212,  0.1994,  0.9797],
        [-0.1755, -0.2294,  0.9574]], dtype=torch.float64)
torch.Size([5, 3])
f2 =  tensor([[ 0.2131,  0.2140,  0.9533],
        [ 0.0706,  0.0971,  0.9928],
        [-0.0778,  0.1619,  0.9837],
        [-0.2266,  0.1683,  0.9594],
        [-0.1968, -0.1422,  0.9701]], dtype=torch.float64)
torch.Size([5, 3])
f3 =  tensor([[ 0.0147,  0.1783,  0.9839],
        [ 0.2471, -0.0575,  0.9673],
        [ 0.1339, -0.2092,  0.9687],
        [-0.0962, -0.1063,  0.9897],
        [ 0.1035, -0.1424,  0.9844]], dtype=torch.float64)
torch.Size([5, 3])


3. Création of a solution variable : maximum 4 solutions  

    Matrix (4,3,4)  
    Each layer is a solution, for each leayer : first column stres the camera position matrix C (3,1) and the remaining 3 columns store the rotation matrix R (3,3)

In [52]:
solutions = torch.empty((batch_size,4,3,4), dtype=torch.float64)

print("solutions = \n", solutions)
print(solutions.shape)  # (batch_size,4,3,4)

solutions = 
 tensor([[[[3.1502e-311,  3.6421e-01,  8.3898e-01,  4.0432e-01],
          [ 3.6461e+00,  4.5349e-01,  5.3895e-01,  7.0984e-01],
          [ 2.6571e+00,  8.1345e-01,  7.5173e-02,  5.7676e-01]],

         [[ 2.8263e-08,  1.0000e+00,  5.3871e-09,  5.5948e-09],
          [ 5.0828e-08,  5.3871e-09,  1.0000e+00,  8.9790e-09],
          [ 6.0000e+00,  5.5948e-09,  8.9790e-09,  1.0000e+00]],

         [[ 5.0854e-01,  7.7236e-01,  1.8046e-01,  6.0901e-01],
          [ 7.0638e-02,  6.3400e-01,  2.7770e-01,  7.2176e-01],
          [ 7.5580e-02,  3.8875e-02,  9.4357e-01,  3.2889e-01]],

         [[ 5.0854e-01,  7.7236e-01,  1.8046e-01,  6.0901e-01],
          [ 7.0637e-02,  6.3400e-01,  2.7770e-01,  7.2176e-01],
          [ 7.5580e-02,  3.8875e-02,  9.4357e-01,  3.2889e-01]]],


        [[[ 1.2956e+00,  1.2579e-03,  9.9603e-01,  8.9012e-02],
          [ 3.5113e+00,  8.6718e-01,  4.3241e-02,  4.9611e-01],
          [ 4.0668e+00,  4.9799e-01,  7.7814e-02,  8.6368e-01]],

         [[ 3.

4. Verification that the 3 points given are not collinear 

In [53]:
# Test of non-collinearity
v1 = P2 - P1 # (batch_size, 3)
 
v2 = P3 - P1 # (batch_size, 3)

norms = torch.norm(torch.cross(v1,v2, dim=1),dim = 1 )

all_dif_zero = torch.all(norms != 0)  

if not all_dif_zero:
    print('\nProblem: the points must not be collinear')
else:
    print('\nThe points are not collinear, we can continue')


The points are not collinear, we can continue


5. Creation of an orthonormal frame from f1, f2, f3 (the features vectors)

In [54]:
# Calculation of vectors of the base τ = (C,tx,ty,tz)
tx = f1     # (batch_size,3)
print("tx = ", tx)
print(tx.shape)  # (batch_size,3)

tz = torch.cross(f1,f2,dim=1)/ torch.norm(torch.cross(f1,f2,dim=1),dim=1, keepdim=True) 
# (batch_size,3)
print("tz = ", tz)
print(tz.shape)  # (batch_size,3)

ty = torch.cross(tz,tx,dim=1) # (batch_size,3)
print("ty = ", ty)
print(ty.shape)  # (batch_size,3)

tx =  tensor([[ 0.0437,  0.0462,  0.9980],
        [-0.2392, -0.1993,  0.9503],
        [ 0.2706, -0.1246,  0.9546],
        [-0.0212,  0.1994,  0.9797],
        [-0.1755, -0.2294,  0.9574]], dtype=torch.float64)
torch.Size([5, 3])
tz =  tensor([[-0.7038,  0.7104, -0.0021],
        [-0.6896,  0.7239, -0.0217],
        [-0.6294, -0.7732,  0.0775],
        [ 0.1274, -0.9714,  0.2005],
        [-0.9538, -0.2012, -0.2230]], dtype=torch.float64)
torch.Size([5, 3])
ty =  tensor([[ 0.7090,  0.7023, -0.0636],
        [ 0.6836,  0.6605,  0.3106],
        [-0.7284,  0.6218,  0.2876],
        [-0.9916, -0.1291,  0.0048],
        [-0.2437,  0.9523,  0.1835]], dtype=torch.float64)
torch.Size([5, 3])


5. (bis) Creation of a transformation matrix T and expression of the f3 vector in this frame

In [55]:
tx = torch.reshape(tx,(batch_size,1,3))   # (batch_size,1,3)
print("tx = \n", tx)

ty = torch.reshape(ty,(batch_size,1,3))  # (batch_size,1,3)
print("ty = \n", ty)

tz = torch.reshape(tz,(batch_size,1,3))  # (batch_size,1,3)
print("tz = \n", tz)

# Computation of the matrix T and the feature vector f3
T = torch.cat((tx,ty,tz),dim = 1) # (3*3)
print("T = \n", T)
print(T.shape)  # (batch_size,3,3)


f3_T = torch.matmul(T,f3.unsqueeze(-1)) # (
print("f3_T = \n", f3_T)
print(f3_T.shape)  # (batch_size,3,1)


tx = 
 tensor([[[ 0.0437,  0.0462,  0.9980]],

        [[-0.2392, -0.1993,  0.9503]],

        [[ 0.2706, -0.1246,  0.9546]],

        [[-0.0212,  0.1994,  0.9797]],

        [[-0.1755, -0.2294,  0.9574]]], dtype=torch.float64)
ty = 
 tensor([[[ 0.7090,  0.7023, -0.0636]],

        [[ 0.6836,  0.6605,  0.3106]],

        [[-0.7284,  0.6218,  0.2876]],

        [[-0.9916, -0.1291,  0.0048]],

        [[-0.2437,  0.9523,  0.1835]]], dtype=torch.float64)
tz = 
 tensor([[[-0.7038,  0.7104, -0.0021]],

        [[-0.6896,  0.7239, -0.0217]],

        [[-0.6294, -0.7732,  0.0775]],

        [[ 0.1274, -0.9714,  0.2005]],

        [[-0.9538, -0.2012, -0.2230]]], dtype=torch.float64)
T = 
 tensor([[[ 0.0437,  0.0462,  0.9980],
         [ 0.7090,  0.7023, -0.0636],
         [-0.7038,  0.7104, -0.0021]],

        [[-0.2392, -0.1993,  0.9503],
         [ 0.6836,  0.6605,  0.3106],
         [-0.6896,  0.7239, -0.0217]],

        [[ 0.2706, -0.1246,  0.9546],
         [-0.7284,  0.6218,  0.2876],
  

The sing of the z-coordinate in f3_T give us the sign of teta, that we will need after 

In [56]:
print(f3_T[:,2])
f3_T_positif = f3_T[:,2] > 0        # (batch_size,1)

print("f3_T_positif = \n", f3_T_positif)
print(f3_T_positif.shape)  

tensor([[ 0.1143],
        [-0.2330],
        [ 0.1525],
        [ 0.2894],
        [-0.2897]], dtype=torch.float64)
f3_T_positif = 
 tensor([[ True],
        [False],
        [ True],
        [ True],
        [False]])
torch.Size([5, 1])


6. Change of frame is performed on the 3D points side, and the transformation matrix N is defined

In [57]:
# Calculation of vectors of the base η = (P1,nx,ny,nz)
nx = (P2 - P1)/torch.norm(P2 - P1,dim=1,keepdim=True)      #(batch_size,3)
nz = torch.cross(nx,P3-P1,dim=1)/torch.norm(torch.cross(nx,P3-P1,dim=1), dim=1, keepdim=True)  # (batch_size,3)
ny = torch.cross(nz,nx,dim=1) # (batch_size,3)



# Reshape the vectors to (1,3) for concatenation
nx = torch.reshape(nx,(batch_size,1,3))  # (batch_size,1,3)
ny = torch.reshape(ny,(batch_size,1,3))
nz = torch.reshape(nz,(batch_size,1,3))

print("nx = \n", nx)
print(nx.shape)  # (1*3)
print("ny = \n", ny)
print("nz = \n", nz)

# Computation of the matrix N and the world point P3
N = torch.cat((nx,ny,nz),dim = 1) #  T's equivalent in the world coordinate system
print("N = \n", N)
print(N.shape)  # (batch_size,3,3)

print("P3.shape = ", P3.shape)  # (batch_size,3)

P3_n = torch.matmul(N,(P3-P1).unsqueeze(-1)) 


print("P3_n = \n", P3_n)
print(P3_n.shape)  # (5,3,1)


nx = 
 tensor([[[ 0.7080, -0.7013,  0.0830]],

        [[ 0.5463, -0.5396, -0.6407]],

        [[-0.7748, -0.6320, -0.0128]],

        [[-0.6797, -0.0617, -0.7309]],

        [[ 0.1045, -0.4740,  0.8743]]], dtype=torch.float64)
torch.Size([5, 1, 3])
ny = 
 tensor([[[-0.6666, -0.7025, -0.2492]],

        [[ 0.8370,  0.3228,  0.4418]],

        [[-0.3898,  0.4618,  0.7967]],

        [[ 0.2614,  0.9107, -0.3200]],

        [[ 0.9582, -0.1875, -0.2161]]], dtype=torch.float64)
nz = 
 tensor([[[ 0.2330,  0.1211, -0.9649]],

        [[-0.0316, -0.7776,  0.6279]],

        [[-0.4977,  0.6223, -0.6042]],

        [[ 0.6853, -0.4085, -0.6028]],

        [[ 0.2664,  0.8603,  0.4346]]], dtype=torch.float64)
N = 
 tensor([[[ 0.7080, -0.7013,  0.0830],
         [-0.6666, -0.7025, -0.2492],
         [ 0.2330,  0.1211, -0.9649]],

        [[ 0.5463, -0.5396, -0.6407],
         [ 0.8370,  0.3228,  0.4418],
         [-0.0316, -0.7776,  0.6279]],

        [[-0.7748, -0.6320, -0.0128],
         [-0.3898,

7. Definition of the variables for the following steps 

In [58]:
# Computation of phi1 et phi2 with 0=x, 1=y, 2=z
phi1 = f3_T[:,0]/f3_T[:,2]  # (batch_size,1)
phi2 = f3_T[:,1]/f3_T[:,2]  # (batch_size,1)
print("phi1 = ", phi1)
print("phi2 = ", phi2)

# Extraction of p1 and p2 from P3_eta
p1 = P3_n[:,0] #x  # (batch_size,3)
p2 = P3_n[:,1] #y  # (batch_size,3)
print("p1 = ", p1)
print("p2 = ", p2)

# Computation of d12
d12 = torch.norm(P2-P1,dim=1, keepdim=True)  # (batch_size,1)
print("d12 = ", d12)

# Computation of b = cot(beta)
cosBeta =( torch.sum(f1*f2,dim=1)/(torch.norm(f1,dim=1)*torch.norm(f2,dim=1)) ).unsqueeze(-1)  # tensor.dot(a,b) <=> tensor.sum(a*b)   # (batch_size,1)
print("cosBeta = ", cosBeta)  

b = torch.sqrt(1/(1-cosBeta**2)-1)

b = torch.where(cosBeta < 0, -b, b)  # If cosBeta < 0, then b = -b    # (batch_size,1)
print("b = ", b)


phi1 =  tensor([[ 8.6697],
        [-3.7410],
        [ 6.4721],
        [ 3.2845],
        [-3.3037]], dtype=torch.float64)
phi2 =  tensor([[ 0.6400],
        [-1.8517],
        [ 0.3343],
        [ 0.3933],
        [-0.0684]], dtype=torch.float64)
p1 =  tensor([[0.5388],
        [2.3878],
        [0.6595],
        [1.8709],
        [0.6814]], dtype=torch.float64)
p2 =  tensor([[0.8768],
        [1.7575],
        [1.5342],
        [2.3797],
        [2.1617]], dtype=torch.float64)
d12 =  tensor([[1.7645],
        [3.2844],
        [3.0375],
        [2.4803],
        [2.0100]], dtype=torch.float64)
cosBeta =  tensor([[0.9706],
        [0.9072],
        [0.8978],
        [0.9782],
        [0.9959]], dtype=torch.float64)
b =  tensor([[ 4.0309],
        [ 2.1561],
        [ 2.0390],
        [ 4.7139],
        [10.9968]], dtype=torch.float64)


8. Calculation of the coefficients of the polynomial 

In [59]:
a4 = - phi2**2 * p2**4 - phi1**2 * p2**4 - p2**4
a3 = 2 * p2**3 * d12 * b + 2 * phi2**2 * p2**3 * d12 * b - 2 * phi1 * phi2 * p2**3 * d12
a2 = - phi2**2 * p1**2 * p2**2 - phi2**2 * p2**2 * d12**2 * b**2 - phi2**2 * p2**2 * d12**2 + phi2**2 * p2**4 + phi1**2 * p2 **4 + 2 * p1 * p2**2 * d12 + 2 * phi1 * phi2 * p1 * p2**2 * d12 * b - phi1**2 * p1**2 * p2**2 + 2 * phi2**2 * p1 * p2**2 * d12 - p2**2 * d12**2 * b**2 - 2 * p1**2 * p2**2
a1 = 2 * p1**2 * p2 * d12 * b + 2 * phi1 * phi2 * p2**3 * d12 - 2 * phi2**2 * p2**3 * d12 * b - 2 * p1 * p2 * d12**2 * b
a0 = - 2 * phi1 * phi2 * p1 * p2**2 * d12 * b + phi2**2 * p2**2 * d12**2 + 2 * p1**3 * d12 - p1**2 * d12**2 + phi2**2 * p1**2 * p2**2 - p1**4 - 2 * phi2**2 * p1 * p2**2 * d12 + phi1**2 * p1**2 * p2**2 + phi2**2 * p2**2 * d12**2 * b**2

print("a4 = ", a4)
print(a4.shape)  # (batch_size,1)
print("a3 = ", a3)
print("a3.shape = ", a3.shape)  # (batch_size,1)
print("a2 = ", a2)
print(a2.shape)  # (batch_size,1)
print("a1 = ", a1)
print(a1.shape)  # (batch_size,1)
print("a0 = ", a0)
print(a0.shape)  # (batch_size,1)


a4 =  tensor([[ -45.2513],
        [-175.7923],
        [-238.2347],
        [-383.0204],
        [-260.2734]], dtype=torch.float64)
torch.Size([5, 1])
a3 =  tensor([[3.1745e-01],
        [9.3489e+01],
        [2.2637e+00],
        [2.7753e+02],
        [4.3948e+02]], dtype=torch.float64)
a3.shape =  torch.Size([5, 1])
a2 =  tensor([[    6.3039],
        [  -37.9062],
        [  136.9314],
        [ -424.1807],
        [-2038.7971]], dtype=torch.float64)
torch.Size([5, 1])
a1 =  tensor([[  1.0339],
        [-69.8893],
        [ 12.6661],
        [-25.8327],
        [-79.4259]], dtype=torch.float64)
torch.Size([5, 1])
a0 =  tensor([[4.7841e-02],
        [5.7895e+01],
        [1.0394e+01],
        [1.2575e+01],
        [1.7737e+00]], dtype=torch.float64)
torch.Size([5, 1])


9. Recovery of the polynomial roots cos (teta)

In [60]:
from torch import vmap

# Computation of the roots
roots = polynomial_root_calculation_4th_degree_ferrari(a0,a1,a2,a3,a4) # (batch_size,4)

print("roots = \n", roots)  # list of tensor (for complex numbers)


roots = 
 tensor([[[ 0.4461,  0.0000],
         [ 0.3347,  0.7975],
         [ 0.8412,  0.0000],
         [ 0.3820,  1.0099],
         [ 0.8635,  2.6749]],

        [[-0.2521,  0.0000],
         [ 0.3347, -0.7975],
         [-0.7573,  0.0000],
         [ 0.3820, -1.0099],
         [ 0.8635, -2.6749]],

        [[-0.0935,  0.0257],
         [ 0.5983,  0.0000],
         [-0.0372,  0.2590],
         [ 0.1493,  0.0000],
         [ 0.0159,  0.0000]],

        [[-0.0935, -0.0257],
         [-0.7359, -0.0000],
         [-0.0372, -0.2590],
         [-0.1887, -0.0000],
         [-0.0543, -0.0000]]], dtype=torch.float64)


10. For each solution : computation of the camera position and rotation matrix

In [61]:
# For each solution of the polynomial
for i in range(4):
  # Computation of trigonometrics forms
  cos_teta = (roots[i,:,0]).unsqueeze(-1) # real part of the root (batch_size,1)

  print("cos_teta = \n", cos_teta)   # (batch_size,1)
  sin_teta = torch.where(f3_T_positif,-torch.sqrt(1-cos_teta**2),torch.sqrt(1-cos_teta**2))
  print("sin_teta = \n", sin_teta) # (batch_size,1)

  cot_alpha = ((phi1/phi2)*p1 + cos_teta*p2 -d12*b )/ ((phi1/phi2)*cos_teta* p2 - p1 + d12)

  sin_alpha = torch.sqrt(1/(cot_alpha**2+1))
  cos_alpha = torch.sqrt(1-sin_alpha**2)
  cos_alpha = torch.where(cot_alpha < 0, -cos_alpha, cos_alpha)  

  # Computation of the intermediate rotation's matrixs
  C_estimate = torch.stack([d12*cos_alpha*(sin_alpha*b + cos_alpha), d12*sin_alpha*cos_teta*(sin_alpha*b+cos_alpha), d12*sin_alpha*sin_teta*(sin_alpha*b+cos_alpha)],dim=1)  # (batch_size,3,1)
  # (batch_size,3,1)

  Q_row1 = torch.stack([-cos_alpha, -sin_alpha*cos_teta, -sin_alpha*sin_teta], dim=-1)  
  Q_row2 = torch.stack([sin_alpha, -cos_alpha*cos_teta, -cos_alpha*sin_teta],dim=-1)
  Q_row3 = torch.stack([0*sin_teta, -sin_teta, cos_teta], dim=-1)
  Q = torch.stack([Q_row1,Q_row2 ,Q_row3],dim=1).squeeze(2)  # (batch_size,3*3)
  


  # Computation of the absolute camera center
  C_estimate = P1.unsqueeze(-1) + torch.matmul(torch.transpose(N, 1,2), C_estimate) # (batch_size,3,1)
 
  

  # Computation of the orientation matrix
  R_estimate = torch.matmul(torch.matmul(torch.transpose(N,1,2),torch.transpose(Q, 1,2)),T)   # (batch_size,3,3)

  # Adding C and R to the solutions
  solutions[:,i,:,:1]= C_estimate
  solutions[:,i,:,1:] = R_estimate

print("solutions = \n", solutions)


cos_teta = 
 tensor([[0.4461],
        [0.3347],
        [0.8412],
        [0.3820],
        [0.8635]], dtype=torch.float64)
sin_teta = 
 tensor([[-0.8950],
        [ 0.9423],
        [-0.5407],
        [-0.9242],
        [ 0.5044]], dtype=torch.float64)
cos_teta = 
 tensor([[-0.2521],
        [ 0.3347],
        [-0.7573],
        [ 0.3820],
        [ 0.8635]], dtype=torch.float64)
sin_teta = 
 tensor([[-0.9677],
        [ 0.9423],
        [-0.6531],
        [-0.9242],
        [ 0.5044]], dtype=torch.float64)
cos_teta = 
 tensor([[-0.0935],
        [ 0.5983],
        [-0.0372],
        [ 0.1493],
        [ 0.0159]], dtype=torch.float64)
sin_teta = 
 tensor([[-0.9956],
        [ 0.8012],
        [-0.9993],
        [-0.9888],
        [ 0.9999]], dtype=torch.float64)
cos_teta = 
 tensor([[-0.0935],
        [-0.7359],
        [-0.0372],
        [-0.1887],
        [-0.0543]], dtype=torch.float64)
sin_teta = 
 tensor([[-0.9956],
        [ 0.6771],
        [-0.9993],
        [-0.9820],
      

In [62]:
print("solutions = \n", solutions)
print(solutions.shape)  # (batch_size,4,3,4)

solutions = 
 tensor([[[[-2.8844e+00,  8.9775e-01,  1.9711e-01,  3.9396e-01],
          [-3.8200e+00, -4.3331e-02, -8.5046e-01,  5.2425e-01],
          [ 4.3487e+00,  4.3838e-01, -4.8771e-01, -7.5496e-01]],

         [[-3.5828e-08,  1.0000e+00,  4.4169e-10,  5.0223e-09],
          [-1.7563e-08,  4.4169e-10, -1.0000e+00,  2.4239e-09],
          [ 6.0000e+00,  5.0223e-09, -2.4239e-09, -1.0000e+00]],

         [[ 2.5717e+00,  9.1977e-01, -5.4801e-02, -3.8860e-01],
          [-3.6337e+00,  1.8058e-01, -8.2005e-01,  5.4305e-01],
          [ 3.9306e+00, -3.4843e-01, -5.6966e-01, -7.4436e-01]],

         [[ 2.5717e+00,  9.1977e-01, -5.4801e-02, -3.8860e-01],
          [-3.6337e+00,  1.8058e-01, -8.2005e-01,  5.4305e-01],
          [ 3.9306e+00, -3.4843e-01, -5.6966e-01, -7.4436e-01]]],


        [[[-1.7892e+00,  9.1474e-01, -2.2274e-01,  3.3709e-01],
          [ 5.4519e-01, -2.0166e-01, -9.7466e-01, -9.6795e-02],
          [ 4.3050e+00,  3.5011e-01,  2.0563e-02, -9.3648e-01]],

         [[-1.

11. Reprojection of points in 2D from the newly estimated matrices to verify the estimation error.

In [63]:
def projection3D2D(point3D,C,R,A) :
  # 3D point = [ Xw, Yw, Zw ]'   (1*3)
  # T : camera translation matrix : (3*1)
  # R : camera rotation matrix : (3*3)
  # A : intraseca matrix of the camera : (3*3)
  # Output : return the coordonates of the point in 2D 

  PI = torch.cat((torch.eye(3, dtype=torch.float64),torch.zeros((3,1), dtype=torch.float64)),dim=1)  # (3*4)

  Rt = torch.cat((R,C),dim=1)               # (3*4)
  Rt = torch.cat((Rt,torch.tensor([[0,0,0,1]], dtype=torch.float64)),dim=0)   # (4*4)

  point3D_bis = torch.cat((torch.reshape(point3D,(3,1)),torch.tensor([[1]],dtype=torch.float64)),dim=0)   #(4*1)
  
  point2D = torch.tensordot(torch.tensordot(torch.tensordot(A,PI,dims=1),Rt,dims=1),point3D_bis,dims=1)  # 2D point = [u, v, w] (3*1)
  point2D = point2D / point2D[2]        # 2D point = [u, v, 1] (3*1)
  return point2D[:2]


C_transpose = C.unsqueeze(-1) # (batch_size,3,1)

p1 = vmap(projection3D2D)(P1,C_transpose,R,A)   # (batch_size,2,1)
print("p1 = ", p1)
print(p1.shape)  # (batch_size,2,1)
p2 = vmap(projection3D2D)(points3D[1],C_transpose,R,A)
print("p2 = ", p2)
p3 = vmap(projection3D2D)(points3D[2],C_transpose,R,A)
print("p3 = ", p3)
p4 = vmap(projection3D2D)(P4,C_transpose,R,A)
print("p4 = ", p4)




p1 =  tensor([[[355.0230],
         [277.0616]],

        [[118.6493],
         [ 72.1936]],

        [[546.7488],
         [135.5633]],

        [[302.6588],
         [402.8269]],

        [[173.3745],
         [ 48.3374]]], dtype=torch.float64)
torch.Size([5, 2, 1])
p2 =  tensor([[[498.8535],
         [419.5653]],

        [[376.9246],
         [318.2144]],

        [[256.7167],
         [371.6710]],

        [[131.0753],
         [380.3234]],

        [[157.6831],
         [122.7444]]], dtype=torch.float64)
p3 =  tensor([[[331.9615],
         [385.0204]],

        [[524.3531],
         [192.4765]],

        [[430.6045],
         [ 67.2209]],

        [[242.2633],
         [154.1080]],

        [[404.1475],
         [124.3039]]], dtype=torch.float64)
p4 =  tensor([[[255.6398],
         [143.9373]],

        [[496.5206],
         [157.0087]],

        [[101.2832],
         [478.5211]],

        [[ 89.2701],
         [266.3238]],

        [[112.3637],
         [146.1732]]], dtype=torch

12. Calculation of errors = distance between the 2D points estimated from the found rotation and position matrices and the 2D points from the initial matrices

In [69]:
def distance(pt, pt_estimation):
  # input : pt = [u,v] 2D point in the image  (2,1)
  #         pt_estimation = [u_est,v_est] 2D point estimated by P3P (2,1)  

  erreur = torch.tensor(0, dtype=torch.float64)  # Initialize error as a tensor
  for i in range(len(pt)): 
    erreur = erreur + (pt[i] - pt_estimation[i])**2  
  return torch.sqrt(erreur)


def affichage_erreur(solutions,points2D,points3D,A) : 
  
  # Compute the error of estimation for each points after the P3P algorithm 

  # solutions : solution matrix returned by P3P (4*3*4)
  # points 3D : 4 pts 3D used for P3P - list of tensors (batch_size, 3)
  # points 2D : 4 pts 2D used for P3P (image of the 3D points)
  batch_size_ = solutions.shape[0]

  for k in range(batch_size_):  # Iterate over the batch size
    print("\n\n")
    print("============ Batch number : ", k+1,"=====================")

    P1 = points3D[0][k]
    P2 = points3D[1][k]
    P3 = points3D[2][k]
    P4 = points3D[3][k]
    A_ = A[k,:,:]

    erreurs = []
    nb_sol = 0

    for i in range(4):  # Iterate over the 4 solutions: 
      R = solutions[k,i,:,1:] # (3,3)
      C = solutions[k,i,:,:1] # (3,1)
      
      if (R[0,0] != None) : 
        nb_sol += 1 
        print("------------ Solution n° : ",nb_sol,"----------------")
        print("R = \n",R,)
        print("C = \n",C,)

        
        p1_P3P = projection3D2D(P1,C,R,A_)
        p2_P3P = projection3D2D(P2,C,R,A_)
        p3_P3P = projection3D2D(P3,C,R,A_)
        p4_P3P = projection3D2D(P4,C,R,A_)
        pt_2D_P3P = [p1_P3P,p2_P3P,p3_P3P,p4_P3P]   # (4,2)


        erreurs.append([0])
        for j in range(len(points2D)):
            erreur_pt = distance(points2D[j][k],pt_2D_P3P[j])
            erreurs[i]+=erreur_pt
        
    indice_min = 0
    min = erreurs[0]
    for i in range(1,len(erreurs)) :
      if erreurs[i]<min :
        min = erreurs[i]
        indice_min = i

    R_opti = solutions[k,indice_min,:,1:] 
    C_opti = solutions[k,indice_min,:,:1]
    print("\n------------ Best solution : ----------------")
    print("Solution n° :",indice_min+1,"\n")
    print("R estimé = \n", R_opti,"\n")
    print("C estimé = \n", C_opti, "\n")

affichage_erreur(solutions, [p1, p2, p3, p4], [P1,P2,P3, P4], A)




============ Batch number :  1 =====================
------------ Solution n° :  1 ----------------
R = 
 tensor([[ 0.8977,  0.1971,  0.3940],
        [-0.0433, -0.8505,  0.5242],
        [ 0.4384, -0.4877, -0.7550]], dtype=torch.float64)
C = 
 tensor([[-2.8844],
        [-3.8200],
        [ 4.3487]], dtype=torch.float64)
------------ Solution n° :  2 ----------------
R = 
 tensor([[ 1.0000e+00,  4.4169e-10,  5.0223e-09],
        [ 4.4169e-10, -1.0000e+00,  2.4239e-09],
        [ 5.0223e-09, -2.4239e-09, -1.0000e+00]], dtype=torch.float64)
C = 
 tensor([[-3.5828e-08],
        [-1.7563e-08],
        [ 6.0000e+00]], dtype=torch.float64)
------------ Solution n° :  3 ----------------
R = 
 tensor([[ 0.9198, -0.0548, -0.3886],
        [ 0.1806, -0.8201,  0.5431],
        [-0.3484, -0.5697, -0.7444]], dtype=torch.float64)
C = 
 tensor([[ 2.5717],
        [-3.6337],
        [ 3.9306]], dtype=torch.float64)
------------ Solution n° :  4 ----------------
R = 
 tensor([[ 0.9198, -0.0548, -0.